In [3]:
from tensorflow import keras
import tensorflow as tf
print(tf.__version__)
from pprint import pprint
import tensorflow_datasets as tfds

2.5.0


In [4]:
	def loss(y):
		prior, mu, scale = _parse_outputs(output) 
		distribution = 'MultivariateNormalTriL'

		dist  = getattr(tfp.distributions, distribution)(mu, scale)
		prob  = tfp.distributions.Categorical(probs=prior)
		mix   = tfp.distributions.MixtureSameFamily(prob, dist)

		def impute(mix, y, N):
			return tf.reduce_mean([
				mix.log_prob( tf.where(tf.math.is_nan(y), mix.sample(), y) )
			for _ in range(N)], 0)
		likelihood = mix.log_prob(y)
		return tf.reduce_mean(-likelihood) + tf.add_n([0.])
#		return tf.reduce_mean(-likelihood) + tf.add_n([0.] + self.model.losses)

	def _parse_outputs(self, output):
		n_mix = 5
		n_targets = 5
		prior, mu, scale = tf.split(output, [n_mix, n_mix * self.n_targets, -1], axis=1)
		prior = tf.reshape(prior, shape=[-1, n_mix])
		mu    = tf.reshape(mu,    shape=[-1, n_mix, n_targets])
		scale = tf.reshape(scale, shape=[-1, n_mix, n_targets, n_targets])
		return prior, mu, scale


In [5]:
class MixtureLayer(tf.keras.layers.Layer):

	def __init__(self, n_mix, n_targets, epsilon, **layer_kwargs):
		super(MixtureLayer, self).__init__()
		layer_kwargs.pop('activation', None)

		self.n_mix     = n_mix 
		self.n_targets = n_targets 
		self.epsilon   = tf.constant(epsilon)
		self._layer    = tf.keras.layers.Dense(self.n_outputs, **layer_kwargs)


	@property 
	def layer_sizes(self):
		''' Sizes of the prior, mu, and (lower triangle) scale matrix outputs '''
		sizes = [1, self.n_targets, (self.n_targets * (self.n_targets + 1)) // 2]
		return self.n_mix * np.array(sizes)


	@property 
	def n_outputs(self):
		''' Total output size of the layer object '''
		return sum(self.layer_sizes)


	# @tf.function(experimental_compile=True)
	def call(self, inputs):
		prior, mu, scale = tf.split(self._layer(inputs), self.layer_sizes, axis=1)

		prior = tf.nn.softmax(prior, axis=-1) + tf.constant(1e-9)
		mu    = tf.stack(tf.split(mu, self.n_mix, 1), 1) 
		scale = tf.stack(tf.split(scale, self.n_mix, 1), 1) 
		scale = tfp.math.fill_triangular(scale, upper=False)
		norm  = tf.linalg.diag(tf.ones((1, 1, self.n_targets)))
		sigma = tf.einsum('abij,abjk->abik', tf.transpose(scale, perm=[0,1,3,2]), scale)
		sigma+= self.epsilon * norm
		scale = tf.linalg.cholesky(sigma)

		return tf.keras.layers.concatenate([
			tf.reshape(prior, shape=[-1, self.n_mix]),
			tf.reshape(mu,    shape=[-1, self.n_mix * self.n_targets]),
			tf.reshape(scale, shape=[-1, self.n_mix * self.n_targets ** 2]),
		])


In [6]:
model = keras.models.load_model('./solid_model', custom_objects={ 'loss': loss, 'MixtureLayer': MixtureLayer })

In [7]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWjYXdnzybfylaH1ziXNNZZwAsA9HKI-0BEE_81hnlzKquhmOKwlmPM

Successfully saved authorization token.


In [8]:
BANDS = ['Rrs_443', 'Rrs_492', 'Rrs_560', 'Rrs_665', 'Rrs_704']

In [9]:
import folium
ASSET_ID = 'users/cboros/solid'

image = ee.Image(ASSET_ID)
vis = {
 "bands": ["Rrs_740", "Rrs_560", "Rrs_443" ], "min": 0, "max": 0.03}
map_id = image.getMapId(vis)
map = folium.Map(location=[39.279839, -76.31669])
folium.TileLayer(
  tiles=map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='Prediction for TSS',
).add_to(map)
display(map)

In [20]:
lng_min = -76.60002136230469
lng_max = -76.05133056640625
lat_min = 39.04213333129883
lat_max = 39.625762939453125

scale = image.select('Rrs_443').projection().nominalScale();
print(scale.getInfo())
scale = 80

geometry = ee.Geometry.Polygon([[[lng_min,lat_min],[lng_min,lat_max],
                                [lng_max, lat_max], [lng_max, lat_min],
                                [lng_min, lat_min]]])


minValues = image.reduceRegion(reducer=ee.Reducer.min(),
                                 geometry=geometry, scale=scale
                                 )

maxValues = image.reduceRegion(reducer=ee.Reducer.max(),
                                 geometry=geometry, scale=scale
                                 )

percentiles25 = image.reduceRegion(reducer=ee.Reducer.percentile(ee.List([ee.Number(25.0)])),
                                 geometry=geometry, scale=scale
                                 )

percentiles75 = image.reduceRegion(reducer=ee.Reducer.percentile(ee.List([ee.Number(75.0)])),
                                 geometry=geometry, scale=scale
                                 )

medians = image.reduceRegion(reducer=ee.Reducer.median(),
                                 geometry=geometry, scale=scale 
                                 )

medianImages = medians.toImage();
percentileImages25 = percentiles25.toImage();
percentileImages75 = percentiles75.toImage();

scaleImages = percentileImages75.subtract(percentileImages25);

# transform input image
transformedImages = image.subtract(medianImages).divide(scaleImages);


67.90737046122514


In [21]:
# show transformed image

band = 'Rrs_560'

min = minValues.get(band);
max = maxValues.get(band);

min = min.getInfo();
max = max.getInfo();

p75 = percentiles25.get(band);
p25 = percentiles75.get(band);

p75 = p75.getInfo();
p25 = p25.getInfo();

median = medians.get(band).getInfo();

PALETTE = [
  'ffffd9',
  'edf8b1',
  'c7e9b4',
  '7fcdbb',
  '41b6c4',
  '1d91c0',
  '225ea8',
  '253494',    
  '081d58'
]

vis = {
 "bands": [ band ], "min": (min-median) / (p75 - p25), "max": (max - median) / (p75 - p25), "palette": PALETTE}


map_id = transformedImages.getMapId(vis)
map = folium.Map(location=[39.279839, -76.31669])
folium.TileLayer(
  tiles=map_id['tile_fetcher'].url_format,
  attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
  overlay=True,
  name='Prediction for TSS',
).add_to(map)
display(map)

In [22]:
IMAGE_FILE_PREFIX = 'Image_pixel_demo_'
OUTPUT_BUCKET = 'solid_cboros'
OUTPUT_IMAGE_FILE = 'gs://' + OUTPUT_BUCKET + '/Classified_pixel_demo.TFRecord'

# Specify patch and file dimensions.
image_export_options = {
  'patchDimensions': [128, 128],
  'maxFileSize': 104857600,
  'compressed': True
}

# Setup the task.
image_task = ee.batch.Export.image.toCloudStorage(
  image=transformedImages,
  description='Image Export',
  fileNamePrefix=IMAGE_FILE_PREFIX,
  bucket=OUTPUT_BUCKET,
  scale=60,
  fileFormat='TFRecord',
  region=geometry.toGeoJSON()['coordinates'],
  formatOptions=image_export_options,
)

In [23]:
image_task.start()

In [25]:
pprint(ee.batch.Task.list())

[<Task ZG6RMFTERL42ES5CFFWDPFG5 EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task GMW23D7CGIHXO76GUTCJRMOW EXPORT_IMAGE: Image Export (COMPLETED)>,
 <Task IX6Y2P2JVS76E4TZTMCG2RD7 EXPORT_IMAGE: Image Export (COMPLETED)>]


In [26]:
# Get a list of all the files in the output bucket.
files_list = !gsutil ls 'gs://'{OUTPUT_BUCKET}
# Get only the files generated by the image export.
exported_files_list = [s for s in files_list if IMAGE_FILE_PREFIX in s]

# Get the list of image files and the JSON mixer file.
image_files_list = []
json_file = None
for f in exported_files_list:
  if f.endswith('.tfrecord.gz'):
    image_files_list.append(f)
  elif f.endswith('.json'):
    json_file = f

# Make sure the files are in the right order.
image_files_list.sort()

pprint(image_files_list)
pprint(json_file)

['gs://solid_cboros/Image_pixel_demo_.tfrecord.gz']
'gs://solid_cboros/Image_pixel_demo_.json'


In [27]:
import json
from pprint import pprint

# Load the contents of the mixer file to a JSON object.
json_text = !gsutil cat {json_file}
# Get a single string w/ newlines from the IPython.utils.text.SList
mixer = json.loads(json_text.nlstr)
pprint(mixer)

{'patchDimensions': [128, 128],
 'patchesPerRow': 7,
 'projection': {'affine': {'doubleMatrix': [0.0006022351935022611,
                                            0.0,
                                            -76.60002136230469,
                                            0.0,
                                            0.0004823851694822855,
                                            39.04213333129883]},
                'crs': 'EPSG:4326'},
 'totalPatches': 63}


In [28]:
def print_ds(dataset):
    numpy_ds = tfds.as_numpy(dataset)
    for o in numpy_ds:
        print(o)

In [29]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height]

print(patch_dimensions_flat)

# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32) 
    for k in BANDS
]

# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))
print(image_features_dict)

# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset(image_files_list, compression_type='GZIP')


# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)


#for record in image_dataset.take(1):
#  print(repr(record))


# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
)


#for record in image_dataset.take(2):
#  print(repr(record))


# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

#for record in image_dataset.take(2):
#  print(repr(record))

#image_dataset = image_dataset.map(
#  lambda value:  value * 10
#)


# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)


#for record in image_dataset.take(2):
#  print(repr(record))

#print_ds(image_dataset)


[16384]
{'Rrs_443': FixedLenFeature(shape=[16384], dtype=tf.float32, default_value=None), 'Rrs_492': FixedLenFeature(shape=[16384], dtype=tf.float32, default_value=None), 'Rrs_560': FixedLenFeature(shape=[16384], dtype=tf.float32, default_value=None), 'Rrs_665': FixedLenFeature(shape=[16384], dtype=tf.float32, default_value=None), 'Rrs_704': FixedLenFeature(shape=[16384], dtype=tf.float32, default_value=None)}


In [ ]:
predictions = model.predict(image_dataset, steps=patches, verbose=1)

63/63 [==============================] - 64s 941ms/step


In [ ]:
print('Writing to file ' + OUTPUT_IMAGE_FILE)

In [ ]:
print(len(predictions))

In [ ]:
def floats_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))

writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_FILE)

patch = []
cur_patch = 1
for prediction in predictions:
    patch.append(prediction.tolist())
    if (len(patch[0]) == patch_width * patch_height):
        print('Done with patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
        example = tf.train.Example(
          features=tf.train.Features(
            feature={
              'tss': floats_feature(prediction)
            }
          )
        )
        writer.write(example.SerializeToString())
        patch = []
        cur_path += 1

writer.close()

In [ ]:
!earthengine upload image --asset_id=users/cboros/prediction --pyramiding_policy=mode {OUTPUT_IMAGE_FILE} {json_file}

In [ ]:
ee.batch.Task.list()